# 01 - Renderizando Triângulos

Teremos de converter objetos em um espaço 2D contínuo para um conjunto de pixels organizados em uma matriz. Para desenhar pontos, podemos simplesmente identificar o pixel que contém o ponto e colorir o respectivo ponto; para desenhar linhas, podemos usar o algoritmo de Bresenham; porém, para desenhar polígonos, precisamos identificar inicialmente quais pixels vão ser responsáveis pelo espaço do polígono e para isso teremos de descobrir como fazer esse preenchimento.

In [1]:
# Código base para produzir o grid interativo

import sys

if 'google.colab' in sys.modules:
    !pip install ipycanvas
    from google.colab import output
    output.enable_custom_widget_manager()

from ipycanvas import Canvas
import numpy as np
import ipywidgets as widgets

def cria_grid(d=(8,8), p=[], cor='#FF0000'):
    # Grid size and cell size for a compact display
    rows, cols = d
    cell_size = 32
    label_space = 22  # space for row/col numbers
    canvas_width = cols * cell_size + label_space
    canvas_height = rows * cell_size + label_space

    # Create a blank grid
    grid = np.zeros((rows, cols), dtype=int)

    def draw_grid():
        canvas.clear()
        # Draw column numbers
        canvas.font = '16px sans-serif'
        canvas.fill_style = '#222'
        for j in range(cols):
            canvas.fill_text(str(j), label_space + j*cell_size + cell_size//2 - 6, label_space - 6)
        # Draw row numbers
        for i in range(rows):
            canvas.fill_text(str(i), 2, label_space + i*cell_size + cell_size//2 + 6)
        # Draw grid lines
        for i in range(rows+1):
            canvas.stroke_style = '#bbb'
            canvas.stroke_line(label_space, label_space + i*cell_size, canvas_width, label_space + i*cell_size)
        for j in range(cols+1):
            canvas.stroke_style = '#bbb'
            canvas.stroke_line(label_space + j*cell_size, label_space, label_space + j*cell_size, canvas_height)
        # Draw painted cells
        for i in range(rows):
            for j in range(cols):
                if grid[i, j] == 1:
                    canvas.fill_style = '#1976d2'
                    canvas.fill_rect(label_space + j*cell_size + 1, label_space + i*cell_size + 1, cell_size-2, cell_size-2)
        # Draw start and end points
        canvas.fill_style = cor
        for point in p:
            canvas.fill_rect(label_space + point[1]*cell_size + 1, label_space + point[0]*cell_size + 1, cell_size-2, cell_size-2)

    canvas = Canvas(width=canvas_width, height=canvas_height)
    draw_grid()

    painting = [False]

    def paint_cell(x, y):
        col = int((x - label_space) // cell_size)
        row = int((y - label_space) // cell_size)
        if 0 <= row < rows and 0 <= col < cols:
            if grid[row, col] == 0 and (row, col) not in p:
                grid[row, col] = 1
                draw_grid()

    def on_mouse_down(x, y):
        painting[0] = True
        paint_cell(x, y)

    def on_mouse_up(x, y):
        painting[0] = False

    def on_mouse_move(x, y):
        if painting[0]:
            paint_cell(x, y)

    canvas.on_mouse_down(on_mouse_down)
    canvas.on_mouse_up(on_mouse_up)
    canvas.on_mouse_move(on_mouse_move)

    # Button to clear grid
    def clear_grid(_):
        grid[:, :] = 0
        draw_grid()

    clear_btn = widgets.Button(description='Limpar grade')
    clear_btn.on_click(clear_grid)

    # Center the canvas using a Box with layout
    centered = widgets.HBox([canvas], layout=widgets.Layout(justify_content='center'))
    display(widgets.VBox([centered, clear_btn]))

1. Considere os vértices de um triângulo como informado abaixo, e os pixeis A, B, C

Triângulo:  P0 = (2,11)   P1=(11,7)     P2=(6, 2) 

Pixels:  A = (5, 7)  B=(9, 5)  C=(8, 13)

Realize os cálculos no espaço abaixo para identificar se os pixels A, B e C estão dentro, fora ou sobre a aresta do triângulo (P0, P1, P2). Considere o centro do pixel como ponto de amostra.


2. Desenhe os pontos e verifique se seus cálculos estão coerentes.

In [2]:
cria_grid(d=(16, 16))  # Call the function to display the grid